In [1]:
# OBJECT DETECTOR

import cv2  # OpenCV
import numpy as np

default_confidence = 0.5
default_threshold= 0.6
configfile = "SSD/ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"  # Config File for Trained MobileNet SSD Model
frozen_model = "SSD/frozen_inference_graph.pb"  # Weighhts - derived from tensor flow
labelsPath = "SSD/labels.txt"
model = cv2.dnn_DetectionModel(frozen_model, configfile)  # Model used to Detect Objects

LABELS = open(labelsPath).read().strip().split("\n")

# generate different colors to represent each label
COLORS = np.random.uniform(0, 255, size=(len(LABELS), 3))
def draw_bounding_box(image, classID, confidence, x, y, w, h):
    color = COLORS[classID]
    cv2.rectangle(image, (x, y), (x+w, y+h), color, 2)
    text = "{}: {:.4f}".format(LABELS[classID], confidence)
    cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
        0.5, color, 2)

# Customizing input format provided to our Model
model.setInputSize(320, 320)
model.setInputScale(1.0 / 127.5)
model.setInputMean((127.5, 127.5, 127.5))
model.setInputSwapRB(True)

cap = cv2.VideoCapture(0)
boxes = []
confidences = []
classIDs = []

while True:
    ret, frame = cap.read()
    if ret == True:
        ClassIndex, Confidence, box = model.detect(frame, default_threshold)  # Class Index values start from 1 
        
        for classID, confidence, Boxes in zip(ClassIndex, Confidence, box):
            if classID >=1 and classID <=80:
                boxes.append([Boxes[0], Boxes[1], Boxes[2] - Boxes[0], Boxes[3]- Boxes[1]])
                confidences.append(float(confidence))
                classIDs.append(classID-1)
                # cv2.rectangle(frame, Boxes, (255, 0, 0), 2)  # Boxes overlayed around the Detected object
                draw_bounding_box(frame, classID-1, confidence, Boxes[0], Boxes[1], Boxes[2], Boxes[3])
                # string = str(LABELS[classID-1])  # the Label Displayed in the Image
                # cv2.putText(frame, string, (Boxes[0]+10, Boxes[1]+40), font, fontScale=font_scale, color=(0, 255, 0), thickness=3)
        idices = cv2.dnn.NMSBoxes(boxes, confidences, default_confidence,
        default_threshold)

        # if len(idices) > 0:
        #     for i in idices.flatten():
        #         (x, y) = (boxes[i][0], boxes[i][1])
        #         (w, h) = (boxes[i][2], boxes[i][3])

        #         draw_bounding_box(frame, classIDs[i], confidences[i], x, y, w, h)

        cv2.imshow("OBJECT DETECTION", frame)
        
        if cv2.waitKey(2) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

: 